## Load and Extract Raw Data from API

This notebook extracts pollutant data from an API, preprocesses it, and prepares it for analysis. The data includes various air quality metrics such as PM2.5, PM10, NO2, O3, and CO levels. After fetching the data, we handle any missing or inconsistent values and convert the data into a structured format. Summary statistics provide a quick overview, while visualizations like line plots and bar charts help explore trends and anomalies. This ensures the dataset is clean, consistent, and ready for further analysis or modeling.

In [1]:
import urllib
import json
import pandas as pd
from pandas import DataFrame, Series

import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
# get HTTP Response from url
url = "https://datenhub.ulm.de/ckan/api/3/action/datastore_search?resource_id=b49de35e-040c-4530-9208-eefadc97b610&limit=300000" 
response = urllib.request.urlopen(url)
assert response.code == 200

# convert HTTP Response to json
string = response.read().decode("utf-8")
response_dict = json.loads(string)
assert response_dict["success"] is True

# convert JSON object to dataframe
created_package = response_dict["result"]
df_raw = pd.DataFrame.from_dict(created_package["records"])

# remove \r\n from string conversion
df_raw.replace({r"\r\n": ""}, regex=True, inplace=True)

df_raw

,_id,schadstoff,timestamp,wert,station
0,1,NO2,2020-11-13T00:00:00,14.000,Ulm in der Wanne
1,2,NO2,2020-11-13T01:00:00,14.000,Ulm in der Wanne
2,3,NO2,2020-11-13T02:00:00,14.000,Ulm in der Wanne
3,4,NO2,2020-11-13T03:00:00,15.000,Ulm in der Wanne
4,5,NO2,2020-11-13T04:00:00,14.000,Ulm in der Wanne
...,...,...,...,...,...
260881,260883,PM2.5,2024-04-16T07:00:00,5.400,Ulm in der Wanne
260882,260884,NO2,2024-04-16T09:00:00,10.268,Ulm in der Wanne
260883,260885,O3,2024-04-16T09:00:00,77.680,Ulm in der Wanne
260884,260886,PM10,2024-04-16T09:00:00,4.320,Ulm in der Wanne


In [4]:
# Cleanup the df
df_raw.rename(columns={'_id': 'id', 'schadstoff': 'pollutant', 'wert': 'value'}, inplace=True)
df_filtered = df_raw[df_raw['station'] == "Ulm in der Wanne"].copy()
df_filtered.drop(columns={"id", "station"}, inplace=True)
df_filtered.head()

,pollutant,timestamp,value
0,NO2,2020-11-13T00:00:00,14.0
1,NO2,2020-11-13T01:00:00,14.0
2,NO2,2020-11-13T02:00:00,14.0
3,NO2,2020-11-13T03:00:00,15.0
4,NO2,2020-11-13T04:00:00,14.0


## Inspect and clean dataframe

In [5]:
# Inspect df
min_timestamp = df_filtered['timestamp'].min()
max_timestamp = df_filtered['timestamp'].max()
print(f"Min timestamp: {min_timestamp}")
print(f"Max timestamp: {max_timestamp}")

Min timestamp: 2020-11-13T00:00:00
Max timestamp: 2024-04-16T09:00:00


In [6]:
# Check for gaps in the hourly recorded data
df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])
df_filtered.set_index('timestamp', inplace=True)

# Resample the data to hourly frequency and identify any missing timestamps
df_resampled = df_filtered.groupby('pollutant').resample('h').asfreq().drop(columns="pollutant").reset_index()

In [7]:
# Fill missing values
def fill_gaps(group: DataFrame) -> DataFrame:    
    # Interpolate remaining gaps
    group['value'] = group['value'].interpolate(method='linear')
    # Fill forward
    group['value'] = group['value'].ffill()
    # Fill backward only where forward fill did not fill
    group['value'] = group['value'].bfill()
    return group

# Apply fill_gaps function to each group
df_filled = df_resampled.groupby('pollutant', group_keys=True).apply(fill_gaps, include_groups=True).drop(columns="pollutant")

# Verify if there are still any missing values
missing_timestamps_after_filling = df_filled[df_filled['value'].isna()]
has_gaps_after_filling = not missing_timestamps_after_filling.empty

print(f"Are there any gaps in the hourly recorded data after filling? {'Yes' if has_gaps_after_filling else 'No'}")

# If there are still gaps, display the missing timestamps
if has_gaps_after_filling:
    print("Remaining missing timestamps:")
    print(missing_timestamps_after_filling.index.get_level_values('timestamp').unique())

df_filled.reset_index(inplace=True)

Are there any gaps in the hourly recorded data after filling? No


In [8]:
df_filled.drop(columns="level_1", inplace=True)

In [9]:
# Define pollutant limits
limits = {
    'PM2.5': {'annual': 25},
    'PM10': {'daily': 50, 'annual': 40},
    'NO2': {'hourly': 200, 'annual': 40},
    'O3': {'8hour_max': 120}
}

def resample_data(df, freq, agg_func):
    """Resample the data based on the given frequency and aggregation function."""
    return df.resample(freq, on='timestamp').agg(agg_func).reset_index()

def add_limits(fig, limit_value, limit_label, df_pollutant_resampled):
    """Add horizontal limit lines to the plot."""
    fig.add_shape(type="line", x0=df_pollutant_resampled['timestamp'].min(), x1=df_pollutant_resampled['timestamp'].max(),
                  y0=limit_value, y1=limit_value, line=dict(color="Gray", dash="dash"), name=limit_label)

def count_exceedances(df, limit_value, column='value'):
    """Count the number of times the pollutant value exceeds the limit."""
    return (df[column] > limit_value).sum()

def plot_pollutant(df_pollutant, pollutant, limit_type, freq, agg_func, limit_label):
    """Plot the pollutant data and add limit lines and exceedance counts."""
    df_pollutant_resampled = resample_data(df_pollutant, freq, {'value': agg_func})
    title = f'{pollutant} {limit_type.capitalize()} Mean'
    fig = px.line(df_pollutant_resampled, x='timestamp', y='value', title=title)
    add_limits(fig, limits[pollutant][limit_type], f'{limit_type.capitalize()} Limit', df_pollutant_resampled)
    exceedances = count_exceedances(df_pollutant_resampled, limits[pollutant][limit_type])
    fig.show()
    print(f'{pollutant} exceeds the {limit_type} limit {exceedances} times.')

def plot_ozone(df_pollutant):
    """Special handling for Ozone (O3) to calculate the 8-hour rolling maximum."""
    df_pollutant = df_pollutant.set_index('timestamp')
    df_pollutant['8hour_max'] = df_pollutant['value'].rolling('8H').max()
    df_pollutant_resampled = df_pollutant.reset_index()
    title = f'O3 8-Hour Maximum'
    fig = px.line(df_pollutant_resampled, x='timestamp', y='8hour_max', title=title)
    add_limits(fig, limits['O3']['8hour_max'], '8-Hour Max Limit', df_pollutant_resampled)
    exceedances = count_exceedances(df_pollutant_resampled, limits['O3']['8hour_max'], column='8hour_max')
    fig.show()
    print(f'O3 exceeds the 8-hour maximum limit {exceedances} times.')

# Plot the filled multivariate time series
for pollutant in df_filled['pollutant'].unique():
    df_pollutant = df_filled[df_filled['pollutant'] == pollutant]
    if pollutant == 'PM2.5':
        plot_pollutant(df_pollutant, pollutant, 'annual', 'D', 'mean', 'Annual')
    elif pollutant == 'PM10':
        plot_pollutant(df_pollutant, pollutant, 'daily', 'D', 'mean', 'Daily')
        plot_pollutant(df_pollutant, pollutant, 'annual', 'Y', 'mean', 'Annual')
    elif pollutant == 'NO2':
        plot_pollutant(df_pollutant, pollutant, 'hourly', 'H', 'mean', 'Hourly')
        plot_pollutant(df_pollutant, pollutant, 'annual', 'Y', 'mean', 'Annual')
    elif pollutant == 'O3':
        plot_ozone(df_pollutant)

NO2 exceeds the hourly limit 0 times.


NO2 exceeds the annual limit 0 times.


O3 exceeds the 8-hour maximum limit 17 times.


PM10 exceeds the daily limit 3 times.


PM10 exceeds the annual limit 0 times.


PM2.5 exceeds the annual limit 11 times.


First, there is a noticeable distribution shift in NO2 levels, indicating significant changes over time. Additionally, there is a small gap in the data for PM10 and PM2.5 in March 2024, which needs addressing. Lastly, the values for all pollutants exhibit clear seasonal patterns, suggesting regular fluctuations throughout the year. These insights are crucial for developing targeted strategies to monitor and manage air quality more effectively.

#### Store transformed data as csv

In [12]:
# Define the file path
csv_file = os.path.join('..', 'data', 'transformed', 'pollutant_data_cleaned.csv')
os.makedirs(os.path.dirname(csv_file), exist_ok=True)

# Save the dataframe as a CSV file
df_filled.to_csv(csv_file, index=False)
print(f'Data saved to {csv_file}')

Data saved to ..\data\processed\pollutant_data_cleaned.csv
